# Lab 5: Warping and Image Segmentation

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

imagesDir = '../Images_03a' # Change this, according to your images' directory path

## 1. Warping

In [10]:
# Read image
img = cv2.imread(os.path.join(imagesDir, 'giraffe.jpg')) # Change this, according to your image's path

# Resize image to facilitate visualization
img = cv2.resize(img, (0, 0), fx = 0.4, fy = 0.4)

# Show image
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Affine Transformation

In [7]:
# Select original coordinates of three points of the original image
ori_coord = np.array([[0, 0], [img.shape[1] - 1, 0], [0, img.shape[0] - 1]]).astype(np.float32)

# Select target coordinates (where the points will move to)
tar_coord = np.array([[0, img.shape[1]*0.33], [img.shape[1]*0.85, img.shape[0]*0.25], [img.shape[1]*0.15, img.shape[0]*0.7]]).astype(np.float32)

# Get affine transformation matrix
warp_mat = cv2.getAffineTransform(ori_coord, tar_coord)

# Apply transformation to the image
warp_dst = cv2.warpAffine(img, warp_mat, (img.shape[1], img.shape[0]))

# Show Image
cv2.imshow('Warped Image', warp_dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.1: Rotate an image by [defining rotation matrix](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gafbbc470ce83812914a70abfb604f4326) and then applying transformation to the image.

In [42]:
# Specify the rotation angle
angle = 45  # Rotate 45 degrees clockwise

# Get image dimensions
height, width = img.shape[:2]
print(img.shape)

# Define the rotation matrix
rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

# Apply the rotation transformation
rotated_image = cv2.warpAffine(img, rotation_matrix, (width, height))

# Display the original and rotated images
cv2.imshow('Original Image', img)
cv2.imshow('Rotated Image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(720, 720, 3)


Exercise 1.2: Rotate an image through an Affine Transformation instead of a rotation matrix.

In [15]:
# Select original coordinates of three points of the original image
ori_coord = np.array([[0, 0], [height, 0], [0, width]]).astype(np.float32)

# Select target coordinates (where the points will move to)
tar_coord = np.array([[height, 0], [height, width], [0, 0]]).astype(np.float32)

# Get affine transformation matrix
warp_mat = cv2.getAffineTransform(ori_coord, tar_coord)

# Apply transformation to the image
warp_dst = cv2.warpAffine(img, warp_mat, (img.shape[1], img.shape[0]))

# Show Image
cv2.imshow('Warped Image', warp_dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.3: Apply a translation of 100 pixels to the right.

In [12]:
translation_matrix = np.float32([[1, 0, 100], [0, 1, 0]])  # Translate 100 pixels to the right

# Apply the translation transformation
translated_image = cv2.warpAffine(img, translation_matrix, (width, height))

# Display the original and translated images
cv2.imshow('Original Image', img)
cv2.imshow('Translated Image', translated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Homography

Example of homography using feature matching from last week

In [13]:
# Load images
img1 = cv2.imread(os.path.join(imagesDir, 'match_box01a_1.png'), cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(os.path.join(imagesDir, 'match_box01a_2.png'), cv2.IMREAD_GRAYSCALE)

cv2.imshow('Query', img1)
cv2.imshow('Train', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# Initiate SIFT detector
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# Apply FLAN Matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

# Store all the good matches as per Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

# Stop if no matches are found
if len(good) < 10:
    print( "Error: Insufficient number of matches")
    exit(-1)

# Obtain points corresponding to the matches in the query and train images
query_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
train_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

# Obtain homography that represents the transformation from the points of the train image into the position of the query image 
M, mask = cv2.findHomography(train_pts, query_pts, cv2.RANSAC, 5.0)

# Apply transformation to image
warped_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]),flags=cv2.INTER_LINEAR)

cv2.imshow('Query', img1)
cv2.imshow('Original Image', img2)
cv2.imshow('Warped Image', warped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.4: Draw lines around the object by:
* Obtaining homography that transforms points from the query image to the train image
* Applying [the perspectiveTransform function](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#gad327659ac03e5fd6894b90025e6900a7) to obtain the coordinates of the object of the query image on the train image
* Drawing lines on the train image that connect the coordinates of the object using [polylines](https://docs.opencv.org/3.4/d6/d6e/group__imgproc__draw.html#gaa3c25f9fb764b6bef791bf034f6e26f5)

In [22]:
# Initiate SIFT detector
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# Apply FLAN Matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

# Store all the good matches as per Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

# Stop if no matches are found
if len(good) < 10:
    print( "Error: Insufficient number of matches")
    exit(-1)

# Obtain points corresponding to the matches in the query and train images
query_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
train_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

# Obtain homography that represents the transformation from the points of the train image into the position of the query image 
M, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)

pts = np.float32([[0,0], [0,img1.shape[0]-1], [img1.shape[1]-1, img1.shape[0]-1], [img1.shape[1]-1,0]]).reshape(-1, 1, 2)

dst = cv2.perspectiveTransform(pts, M)

img2_bgr = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

img2_lines = cv2.polylines(img2_bgr, [np.int32(dst)], True, (255, 0, 0), 3, cv2.LINE_AA)

cv2.imshow('Highlight', img2_lines)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. Segmentation

### Thresholding

In [23]:
# Read image
img = cv2.imread(os.path.join(imagesDir, 'sudoku.png'))

# Convert to grayscale
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Show image
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Otsu Thresholding

In [25]:
# Apply global binary threshold
ret, th_global = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# Apply binary threshold with Otsu's method
ret, th_otsu = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Show images
cv2.imshow('Global Threshold', th_global)
cv2.imshow('Otsu Threshold', th_otsu)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.1: Verify the effects of blurring the image using a Gaussian filter, before applying the Otsu thresholding method.

In [26]:
blured = cv2.GaussianBlur(img, (5, 5), 0)
ret, th_otsu = cv2.threshold(blured, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

cv2.imshow('Otsu Threshold', th_otsu)
cv2.waitKey(0)
cv2.destroyAllWindows()

Adaptive Threshold

In [29]:
# Apply adaptive thresholding
th_adaptive = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Show images
cv2.imshow('Adaptive Mean', th_adaptive)
cv2.imshow('Otsu Threshold', th_otsu)
cv2.imshow('Global Threshold', th_global)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.2: Verify the effects of blurring with filters of increasing sizes before applying the adaptive threshold.

In [ ]:
# TODO

### Segmentation with [K-Means](https://docs.opencv.org/master/d5/d38/group__core__cluster.html#ga9a34dc06c6ec9460e90860f15bcd2f88)

In [37]:
# Read image
img2 = cv2.imread(os.path.join(imagesDir, 'home.jpg'))

# Show image
cv2.imshow('Image', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
# Reshape the image and turn its values to float
print(f"Previous shape: {img2.shape}")

reshaped_image = img2.reshape((-1,3))
reshaped_image = np.float32(reshaped_image)

print(f"Current shape: {reshaped_image.shape}")

# Define criteria and number of clusters (k)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
k = 4

# Apply K-means
ret, label, center = cv2.kmeans(reshaped_image, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

Previous shape: (384, 512, 3)
Current shape: (196608, 3)


In [40]:
# Convert back to uint8, and make resulting image
center = np.uint8(center)
result = center[label.flatten()]
result = result.reshape((img2.shape))

# Show image
cv2.imshow('Image', img2)
cv2.imshow('K-Means Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.3: Experiment with different number of clusters.

In [41]:
# Read image
img2 = cv2.imread(os.path.join(imagesDir, 'home.jpg'))

# Reshape the image and turn its values to float
print(f"Previous shape: {img2.shape}")

reshaped_image = img2.reshape((-1,3))
reshaped_image = np.float32(reshaped_image)

print(f"Current shape: {reshaped_image.shape}")

# Define criteria and number of clusters (k)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
k = 16

# Apply K-means
ret, label, center = cv2.kmeans(reshaped_image, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Convert back to uint8, and make resulting image
center = np.uint8(center)
result = center[label.flatten()]
result = result.reshape((img2.shape))

# Show image
cv2.imshow('Image', img2)
cv2.imshow('K-Means Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Previous shape: (384, 512, 3)
Current shape: (196608, 3)


### Segmentation with [GrabCut](https://docs.opencv.org/4.x/d3/d47/group__imgproc__segmentation.html#ga909c1dda50efcbeaa3ce126be862b37f)

In [33]:
# Read image
img = cv2.imread(os.path.join(imagesDir, 'giraffe.jpg')) # Change this, according to your image's path

# Resize image to facilitate visualization
img = cv2.resize(img, (0, 0), fx = 0.6, fy = 0.6)

# Show image
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
# Define image mask for the GrabCut output with same dimensions as the image
mask = np.zeros(img.shape[:2], np.uint8)

# Define the bounding box coordinates with the object of interest: (x, y, width, heigh)
bb = (0, 0, 400, 500)

# Allocate memory for the two arrays that this algorithm internally uses for the segmentation of the foreground and background
bgModel = np.zeros((1, 65), np.float64)
fgModel = np.zeros((1, 65), np.float64)

# Apply GrabCut
(mask, bgModel, fgModel) = cv2.grabCut(img, mask, bb, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)

# All definite background and probable background pixels are set to 0, and all definite foreground and probable foreground pixels are set to 1
output_mask = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD), 0, 1)

# Scale the mask from the range [0, 1] to [0, 255]
output_mask = (output_mask * 255).astype("uint8")

# Apply a bitwise AND to the image using the generated mask by GrabCut to obtain the final result
grabcut_result = cv2.bitwise_and(img, img, mask=output_mask)

# Show result
cv2.imshow('Output Mask', output_mask)
cv2.imshow('GrabCut Result', grabcut_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.4: Select a region of interest in the image (using cv2.selectROI function) for the GrabCut algorithm.

In [ ]:
# TODO